In [ ]:
import pandas as pd
from utils.plotting import plot_time, plot_iter, plot_trajectories

**Read results:**

In [ ]:
# path to csv file with results

TASK = "income"
STATE = "ok"
DATASET = TASK + "_" + STATE
lb = 0.005
alg = "SSG"
alg_param_str = ""
alg += alg_param_str
constraint = "loss"

filename = f"./utils/exp_results/{alg}_" + f"{DATASET}_{lb}.csv"

stats = pd.read_csv(filename)
if alg_param_str.startswith("_mu0"):
    alg = "sslalm_aug"

In [ ]:
stats_train = stats[stats["is_train"] == "train"].drop(["is_train"], axis=1).dropna()
stats_test = stats[stats["is_train"] == "test"].drop(["is_train"], axis=1).dropna()

**Plots w.r.t. time**

In [ ]:
import os

alg_name = "sslalm_aug" if alg.startswith("sslalm_mu0") else alg.split("_")[0]
os.makedirs(os.path.dirname(f"./plots/{alg}/{DATASET}/"), exist_ok=True)

In [ ]:
f1tr_time, f2tr_time = plot_time(
    stats_train, lb, round_step=0.0005, f_ylim=(0.4, 0.77), c_ylim=(-0.12, 0.12)
)

In [ ]:
f1tr_time.savefig(
    f"./plots/{alg.removesuffix(alg_param_str)}/{DATASET}/loss_train_time_{DATASET}"
)
f2tr_time.savefig(
    f"./plots/{alg.removesuffix(alg_param_str)}/{DATASET}/c_train_time_{DATASET}"
)
print(f"./plots/{alg.removesuffix(alg_param_str)}/{DATASET}")

In [ ]:
f1t_time, f2t_time = plot_time(
    stats_test, lb, round_step=0.0005, f_ylim=(0.4, 0.77), c_ylim=(-0.12, 0.1)
)

In [ ]:
f1t_time.savefig(
    f"./plots/{alg.removesuffix(alg_param_str)}/{DATASET}/loss_test_time_{DATASET}"
)
f2t_time.savefig(
    f"./plots/{alg.removesuffix(alg_param_str)}/{DATASET}/c_test_time_{DATASET}"
)
print(f"./plots/{alg.removesuffix(alg_param_str)}/{DATASET}")

**Plots w.r.t. iteration**

In [ ]:
stats_train[["trial", "iteration"]].groupby(by="trial").max().mean()

In [ ]:
f1tr, f2tr = plot_iter(stats_train, lb, "iteration")

In [ ]:
f1t, f2t = plot_iter(stats_test, lb, "iteration")

**Trajectories:**

In [ ]:
plot_trajectories(stats_train, lb, "iteration", alpha=1)